In [ ]:
# Import pandas, numpy and matplotlib libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import seaborn as sns
from matplotlib.backends.backend_pdf import PdfPages
%matplotlib inline

# Convert csv into dataframe. 
df = pd.read_csv('CreditRisk-6488587.csv',  keep_default_na=True, sep=',\s+', delimiter=',')

In [ ]:
# Allow showing of all columns
pd.set_option('display.max_columns', 30)

# (1) Prepare a data quality report for the CSV file.

### - Check how many rows and columns are in the CSV.

In [ ]:
#Print (rows, features)
print("The number of rows and features:",df.shape)

There are 1000 customers with 24 features associated with each customer.

### - Print the first and the last 5 rows.

In [ ]:
df.head(5)

In [ ]:
df.tail(5)

### - Convert the features to their appropriate data types (e.g., decide which features are more appropriate as continuos and which ones as categorical types). 

In [ ]:
df.dtypes

Based on examining the data in a spreadsheet program, the following decision was made about the data convertions:

- The 'MaxDelq2PublicRecLast12M' and 'MaxDelqEver' columns were converted to **categorical** as they contained finite set of possible values. The numerical values have special meaning, and thus are not continuous in nature.

In [ ]:
# Select columns containing categorical data
categorical_columns = df[['RiskPerformance', 'MaxDelq2PublicRecLast12M', 'MaxDelqEver']].columns
#income was chosen as category because the data contains single digit values representing income categories

# Convert data type to category for these columns
for column in categorical_columns:
    df[column] = df[column].astype('category')

df.dtypes

In [ ]:
# Print table with descriptive statistics for all the categorical features
df.select_dtypes(['category']).describe().T

### - Drop duplicate rows and columns, if any.

In [ ]:
# Examine the duplicate rows
df["is_duplicate"] = df.duplicated()
df[df.duplicated() == True]

In [ ]:
# Check for duplicate rows
print('Duplicate rows:', df.duplicated()[df.duplicated() == True].shape[0])
# Check for duplicate columns
print('Duplicate columns:',df.columns.size - df.columns.unique().size)

### - There are 42 duplicate rows in the dataframe, with 0 duplicate columns. Therefore, the duplicate rows will be dropped.

In [ ]:
df.drop_duplicates(keep=False, inplace=True)

In [ ]:
# Check for duplicate rows again
print('Duplicate rows:', df.duplicated()[df.duplicated() == True].shape[0])

print('Duplicate columns:',df.columns.size - df.columns.unique().size)

In [ ]:
# Drop the 'is_duplicate column'
df = df.drop('is_duplicate',1)

### - Drop constant columns, if any.

We've already seen above that categorical features don't have constant columns. Let's check continuous features:

In [ ]:
# Select columns containing continuous data
continuous_columns = df[['ExternalRiskEstimate','MSinceOldestTradeOpen','MSinceMostRecentTradeOpen','AverageMInFile','NumSatisfactoryTrades', 'NumTrades60Ever2DerogPubRec',
                         'NumTrades90Ever2DerogPubRec', 'PercentTradesNeverDelq', 'MSinceMostRecentDelq', 'NumTotalTrades',  
                         'NumTradesOpeninLast12M', 'PercentInstallTrades', 'MSinceMostRecentInqexcl7days', 'NumInqLast6M',
                         'NumInqLast6Mexcl7days', 'NetFractionRevolvingBurden', 'NetFractionInstallBurden',
                         'NumRevolvingTradesWBalance', 'NumInstallTradesWBalance', 'NumBank2NatlTradesWHighUtilization',
                        'PercentTradesWBalance']].columns

# Print table
df[continuous_columns].describe().T

A non zero **std** or **standard deviation** implies that a particular feature does not contain a single constant value in all of the rows. Thus in this case, none of the continuous features are constant.

### - Save your updated/cleaned data frame to a new csv file.

In [ ]:
# Write the updated dataframe to a csv file
df.to_csv('06488587_cleaned-1.csv', index=False)

### - Prepare a table with descriptive statistics for all the continuous features.

In [ ]:
df.select_dtypes(['int64']).describe().T

### - Prepare a table with descriptive statistics for all the categorical features.

In [ ]:
# Print table with descriptive statistics for all the categorical features
df.select_dtypes(['category']).describe().T

### - Plot histograms for all the continuous features.

In [ ]:
# Plot a histogram of the continuous features
df[continuous_columns].hist(figsize=(25,25))
plt.savefig('06488587_continuous_histograms.pdf')

### - Plot box plots for all the continuous features.

In [ ]:
# Plot box plots for all the continuous features
pp = PdfPages('06488587_continuous_boxplots.pdf')

for col in continuous_columns:
    f = df[col].plot(kind='box', figsize=(10,5))
    pp.savefig(f.get_figure())
    plt.show()

pp.close()

### - Plot bar plots for all the categorical features.

In [ ]:
# Plot bar charts for all categorical features
pp = PdfPages('0648857_categorical_barcharts.pdf')

for column in categorical_columns:
    f = df[column].value_counts().plot(kind='bar', title=column, figsize=(12,10))
    pp.savefig(f.get_figure())
    plt.show()

pp.close()

The Data Quality Report discussion can be found in a pdf file called **06488587_Data_Quality_Report_Initial_Findings.pdf**

# (2) Prepare a data quality plan for the cleaned CSV file.

### The initial list of issues as identified in the **Data_Quality_Report_Initial_Findings.pdf**:

- There are -7, -8, and -9 values in the dataframe which have special meaning, but need to be dealt with, otherwise they will affect the data.
- Many rows have a larger value for 'NumSatisfactoryTrades' than for 'NumTotalTrades'
- 'MaxDelq2PublicRecLast12M' has two values with equal meaning: '5' and '6' both mean unknown delinquency.
- 'NumInqLast6M' and 'NumInqLast6Mexcl7days' both have very similar data.
- 'NumTrades60Ever2DerogPubRec' and 'NumTrades90Ever2DerogPubRec' both have very similar data.
- The outliers in the box plots initially appear to make sense but should be further investigated here.

### In addition to the problems above, some standard checks will be carried out to find any additional issues:

### Check for Irregular cardinalities

In [ ]:
# Check for irregular cardinality in categorical features. There could be same values spelled differently
print("Unique values for:\n- MaxDelqEver:", pd.unique(df.MaxDelqEver.ravel()))
print("\n- MaxDelq2PublicRecLast12M:", pd.unique(df.MaxDelq2PublicRecLast12M.ravel()))
print("\n- RiskPerformance:", pd.unique(df.RiskPerformance.ravel()))

#### There are no irregular cardinalities.

### Null values

In [ ]:
# Check whether there are null values in the data where values would be expected
df.isnull().sum()

#### There are no null values.

### - Proposed solutions to deal with the problems identified:

As data collection is an expensive and/or time consuming process, it is the first priority to always try to keep as much original data intact as possible before making any cuts/removals or modifications. Thus my decisions below are based on this principle of data preservation:

1) **There are -7, -8, and -9 values in the dataframe:**
These values have special meaning, but as we can gather from the data dictionary, -9 No Bureau Record or No Investigation and -8 No Usable/Valid Trades or Inquiries can be treated as missing data, as the meaning behind them doesn't affect our target. Imputation will be performed on these values if there are roughly 30% or less of these values in a row/feature, otherwise the row/feature may need to be dropped.

As for -7, a new binary feature will be created to keep a record of whether a row had this particular value (Condition not Met (e.g. No Inquiries, No Delinquencies)), as it has a special and useful meaning. The -7 values will then be replaced with NaN (not a number) values, so as not to skew the data.


2) **Many rows have a larger value for 'NumSatisfactoryTrades' than for 'NumTotalTrades':**

The data will be investigated to see if the correct values can be found, and if not, the rows will be dropped.


3) **'MaxDelq2PublicRecLast12M' has two values with equal meaning: '5' and '6' both mean unknown delinquency.**

The feature will use only one value to express 'unknown delinquency', with one value being replaced by the other.

4) **'NumInqLast6M' and 'NumInqLast6Mexcl7days' both have very similar data.**
The features will be investigated, and if they are roughly 90%+ similar, one will be dropped (investigation into which will be done).

5) **'NumTrades60Ever2DerogPubRec' and 'NumTrades90Ever2DerogPubRec' both have very similar data.**

The features will be investigated, and if they are roughly 90%+ similar, one will be dropped (investigation into which will be done).


6) **The outliers in the box plots initially appear to make sense but should be further investigated here.**

The outliers will be investigated. If they make sense they will be kept, otherwise the otherlier rows will be removed.

### Applying solutions to the data quality issues:

1) **There are -7, -8, and -9 values in the dataframe:** These values have special meaning, but -9 No Bureau Record or No Investigation and -8 No Usable/Valid Trades or Inquiries can be treated as missing data, as the meaning behind them doesn't affect our target. Imputation will be performed on these values if there are roughly 30% or less of these values in a row/feature, otherwise the row/feature may need to be dropped.

As for -7, a new binary feature will be created to keep a record of whether a row had this particular value (Condition not Met (e.g. No Inquiries, No Delinquencies)), as it has a special and useful meaning. The -7 values will then be replaced with NaN (not a number) values, so as not to skew the data.

In [ ]:
# Find -7 values
df.isin([-7]).sum()

In [ ]:
# Find -8 values
df.isin([-8]).sum()

In [ ]:
# Find -9 values
df.isin([-9]).sum()

### There are two rows which have minus -9 values for each feature. They will be found, and then dropped, as they are not useful for our predictions.

In [ ]:
df[df['ExternalRiskEstimate'] == -9]

In [ ]:
# Drop the rows
df.drop([df.index[13], df.index[50]], inplace=True)


In [ ]:
# Make sure there are no more -9 values
df.isin([-9]).sum()

In [ ]:
# Add a new column called conditionNotMet to record -7 values.
df['ConditionNotMet'] = df.isin([-7]).any(1).astype('category')
# Replace with NaN
df.replace([-7],np.NaN, inplace=True)
df

In [ ]:
# Add a new column to check the number of -8 values per row.
df['NumMinus8']=df.T.isin([-8]).sum()
df

In [ ]:
df.shape

#### There are 26 features, but one is the target, one is 'NumMinus8', and one is 'ConditionNotMet', so 23 features remain. 
#### If there are 11 or more -8 values in a row (50% of the values), we will drop these rows.

In [ ]:
df[df['NumMinus8'] >= 11]

In [ ]:
#No rows need to be dropped.

In [ ]:
df[df['NumMinus8']==df['NumMinus8'].max()]

#### Max value is 7 -8s in 23 features, so roughly 30%. We will use imputation.

In [ ]:
# Get continuous features again after -9s and -7s have been removed.

new_continuous_columns = df[['ExternalRiskEstimate','MSinceOldestTradeOpen','MSinceMostRecentTradeOpen','AverageMInFile','NumSatisfactoryTrades', 
                         'NumTrades90Ever2DerogPubRec', 'PercentTradesNeverDelq', 'NumTotalTrades',  
                         'NumTradesOpeninLast12M', 'PercentInstallTrades', 'MSinceMostRecentInqexcl7days',
                         'NumInqLast6Mexcl7days', 'NetFractionRevolvingBurden', 'NetFractionInstallBurden',
                         'NumRevolvingTradesWBalance', 'NumInstallTradesWBalance', 'NumBank2NatlTradesWHighUtilization',
                        'PercentTradesWBalance', 'MSinceMostRecentDelq']].columns

# Print table
df[new_continuous_columns].describe().T

In [ ]:
# Mean with -8 values remaining
df[new_continuous_columns].mean()

In [ ]:
# Mean with -8 values replaced with NaN, for more accurate values.
df[new_continuous_columns].replace([-8],np.NaN,).mean()

In [ ]:
df[new_continuous_columns].isin([-8]).sum()

### The -8 values will undergo imputation. No features need to be removed as the highest amount of missing values is 31% in 'NetFractionInstallBurden'.

In [ ]:
# Mean or median depending on distribution. Mean is used where there is an existing normal
# distribution. http://datamining.togaware.com/survivor/Mean_Median_Mode.html
df['MSinceOldestTradeOpen'] = df['MSinceOldestTradeOpen'].replace([-8],np.NaN)
df['MSinceOldestTradeOpen'] = df['MSinceOldestTradeOpen'].replace([np.NaN], df['MSinceOldestTradeOpen'].mean())

df['MSinceMostRecentInqexcl7days'] = df['MSinceMostRecentInqexcl7days'].replace([-8], df['MSinceMostRecentInqexcl7days'].median())
df['NetFractionRevolvingBurden'] = df['NetFractionRevolvingBurden'].replace([-8], df['NetFractionRevolvingBurden'].median())
df['NetFractionInstallBurden'] = df['NetFractionInstallBurden'].replace([-8], df['NetFractionInstallBurden'].median())
df['NumRevolvingTradesWBalance'] = df['NumRevolvingTradesWBalance'].replace([-8], df['NumRevolvingTradesWBalance'].median())
df['NumInstallTradesWBalance'] = df['NumInstallTradesWBalance'].replace([-8], df['NumInstallTradesWBalance'].median())
df['NumBank2NatlTradesWHighUtilization'] = df['NumBank2NatlTradesWHighUtilization'].replace([-8], df['NumBank2NatlTradesWHighUtilization'].median())
df['PercentTradesWBalance'] = df['PercentTradesWBalance'].replace([-8], df['PercentTradesWBalance'].median())
df['MSinceMostRecentDelq'] = df['MSinceMostRecentDelq'].replace([-8], df['MSinceMostRecentDelq'].median())

In [ ]:
# Check all -8 values are gone.
df[new_continuous_columns].isin([-8]).sum()

In [ ]:
# Remove the 'NumMinus8' column
df = df.drop('NumMinus8',1)

In [ ]:
# Check min values for minus values.
df.describe().T

2) **Many rows have a larger value for 'NumSatisfactoryTrades' than for 'NumTotalTrades':**

The data will be investigated to see if the correct values can be found, and if not, the rows will be dropped.

In [ ]:
df[['NumSatisfactoryTrades','NumTotalTrades']].sort_values('NumTotalTrades').head(25)

In [ ]:
# Find the rows where 'NumSatisfactoryTrades' is greater than 'NumTotalTrades'
df[df['NumSatisfactoryTrades'] > df['NumTotalTrades']]

#### Take row 27 for example. 
#### MaxDelqEver = 8, which means "current and never delinquent".
#### NumSatisfactoryTrades = 16
#### PercentTradeNeverDelq = 100
#### NumTotalTrades = 6

#### We can see that the MaxDelqEver and PercentTradeNeverDelq match up, but the issue is with the other two values. If we  knew one of either NumSatisfactoryTrades or NumTotalTrades was the true value, then the other could be replaced. For example, if NumSatisfactoryTrades was confirmed to be 16, and PercentTradeNeverDelq = 100, then NumTotalTrades would equal 16. However, if NumTotalTrades was confirmed as 6, then NumSatisfactoryTrades would be replaced with 6. There is no way to determine which is correct, and as only 67 rows are affected, the decision to drop this data has been made. 


In [ ]:
df.drop(df[df['NumSatisfactoryTrades'] > df['NumTotalTrades']].index, inplace=True)

In [ ]:
df[df['NumSatisfactoryTrades'] > df['NumTotalTrades']]

3) **'MaxDelq2PublicRecLast12M' has two values with equal meaning: '5' and '6' both mean unknown delinquency.**

The feature will use only one value to express 'unknown delinquency', with one value being replaced by the other.

In [ ]:
df['MaxDelq2PublicRecLast12M'].unique()

In [ ]:
# 5 and 6 have equal meaning, so it will be set them to one value.
df['MaxDelq2PublicRecLast12M'].replace(5, 6, inplace=True)



4) **'NumInqLast6M' and 'NumInqLast6Mexcl7days' both have very similar data.** 

The features will be investigated, and if they are roughly 90%+ similar, one will be dropped (investigation into which will be done).

5) **'NumTrades60Ever2DerogPubRec' and 'NumTrades90Ever2DerogPubRec' both have very similar data.**

The features will be investigated, and if they are roughly 90%+ similar, one will be dropped (investigation into which will be done).

In [ ]:
# Check for correlation
df.corr(method = 'pearson')

#### 'NumInqLast6M' will be dropped, as 'NumInqLast6Mexcl7days'	excludes the last 7 days removes inquiries that are likely due to price comparision shopping, thus giving potentially more accurate data, and there is a 99% correlation between the features.


In [ ]:
# 99%
df = df.drop('NumInqLast6M', 1)

#### 'NumTrades60Ever2DerogPubRec' will be dropped due an 88% correlation with 'NumTrades90Ever2DerogPubRec'. As 'NumTrades90Ever2DerogPubRec' captures the worse scenario of a customer being 90 days above late on payments, it will be kept.

In [ ]:
# 88%
df = df.drop('NumTrades60Ever2DerogPubRec', 1)

6) **The outliers in the box plots initially appear to make sense but should be further investigated here.**

The outliers will be investigated. If they make sense they will be kept, otherwise the otherlier rows will be removed.

The columns with outliers remaining after the data problem solutions above are:

- MSinceOldestTradeOpen
- MSinceMostRecentTradeOpen
- AverageMInFile
- NumSatisfactoryTrades
- NumTrades90Ever2DerogPubRec
- PercentTradesNeverDelq
- MSinceMostRecentDelq
- NumTotalTrades
- NumTradesOpeninLast12M
- PercentInstallTrades
- MSinceMostRecentInqexcl7days
- NumInqLast6Mexcl7days
- NetFractionRevolvingBurden
- NumRevolvingTradesWBalance
- NumInstallTradesWBalance
- NumBank2NatlTradesWHighUtilization
- PercentTradesWBalance

In [ ]:
df.sort_values(by='MSinceMostRecentTradeOpen', axis=0, ascending=False, inplace=False, kind='quicksort', na_position='last')[['MSinceMostRecentTradeOpen','NumTotalTrades', 'MSinceMostRecentInqexcl7days', 'MSinceOldestTradeOpen', 'AverageMInFile', 'ExternalRiskEstimate', 'NumSatisfactoryTrades', 'NumTrades90Ever2DerogPubRec', 'PercentTradesNeverDelq', 'NumTotalTrades', 'NumTradesOpeninLast12M', 'NumInqLast6Mexcl7days']].head(10)

In [ ]:
df.sort_values(by='MSinceMostRecentTradeOpen', axis=0, ascending=False, inplace=False, kind='quicksort', na_position='last')[['MSinceMostRecentTradeOpen','NumTotalTrades', 'MSinceMostRecentInqexcl7days', 'MSinceOldestTradeOpen', 'AverageMInFile', 'NumTradesOpeninLast12M']].head(10)

In [ ]:
df.sort_values(by='PercentTradesWBalance', axis=0, ascending=False, inplace=False, kind='quicksort', na_position='last')[['NetFractionRevolvingBurden', 'NetFractionInstallBurden','NumRevolvingTradesWBalance', 'NumBank2NatlTradesWHighUtilization', 'NumInstallTradesWBalance']].head(10)

#### Looking at the outliers, there doesn't seem to be anything obviously out of the ordinary; all data values seem to be possible. However, there are so many outliers that it is difficult to ascertain for certain whether or not a specific outlier is wrong or not. For these reasons, the outliers will be left intact.

### Summary of data quality plan:
   
| Feature                 | Data Quality Issue   | Handling Strategy            |
|-------------------------|----------------------|------------------------------|
| MSinceMostRecentInqexcl7days   | -7 special value | Set -7s to NaN, and record -7 presence in new binary column  |                |
| MSinceMostRecentDelq   | -7 special value | Set -7s to NaN, and record -7 presence in new binary column  |
| MSinceOldestTradeOpen   | -8 special value | Imputation |
| MSinceMostRecentDelq   | -8 special value | Imputation |
| MSinceMostRecentInqexcl7days | -8 special value | Imputation |
| NetFractionRevolvingBurden  | -8 special value | Imputation |
| NetFractionInstallBurden   | -8 special value | Imputation |
| NumRevolvingTradesWBalance   | -8 special value | Imputation |
| NumInstallTradesWBalance    | -8 special value | Imputation |
| NumBank2NatlTradesWHighUtilization   | -8 special value | Imputation |
| PercentTradesWBalance   | -8 special value | Imputation |
| All features except RiskPerformance target feature  | -9 special values | Remove affected rows |
| NumSatisfactoryTrades  | Higher values than NumTotalTrades| Remove affected rows  |
| MaxDelq2PublicRecLast12M | Special values with same meaning | Replace value |
| NumTrades60Ever2DerogPubRec | Redundancy with NumTrades90Ever2DerogPubRec| Remove column                |
| NumInqLast6M             | Rendundancy with NumInqLast6Mexcl7days | Remove  column        |
| MSinceOldestTradeOpen                 | Outliers             | Do nothing                   |
| MSinceMostRecentTradeOpen      | Outliers             | Do nothing                   |
| AverageMInFile       | Outliers             | Do nothing                   |
| NumSatisfactoryTrades            | Outliers             | Do nothing                   |
| NumTrades90Ever2DerogPubRec    | Outliers             | Do nothing                   |
|PercentTradesNeverDelq     | Outliers             | Do nothing                   |
| MSinceMostRecentDelq         | Outliers             | Do nothing                   |
| NumTotalTrades             | Outliers             | Do nothing                   |
| NumTradesOpeninLast12M              | Outliers             | Do nothing                   |
| PercentInstallTrades     | Outliers             | Do nothing                   |
| MSinceMostRecentInqexcl7days         | Outliers             | Do nothing                   |
| NumInqLast6Mexcl7days | Outliers           | Do nothing                   |
| NetFractionRevolvingBurden                | Outliers             | Do nothing                   |
| NumRevolvingTradesWBalance | Outliers          | Do nothing                   |
|NumInstallTradesWBalance       | Outliers             | Do nothing                   |
| NumBank2NatlTradesWHighUtilization | Outliers          | Do nothing                   |
| PercentTradesWBalance   | Outliers             | Do nothing                   |

In [ ]:
# Print table with descriptive statistics for all the continuous features
df.select_dtypes(['float64', 'int64']).describe().T

In [ ]:
# Print table with descriptive statistics for all the categorical features
df.select_dtypes(['category']).describe().T

In [ ]:
# Write the dataframe to a csv file
df.to_csv('06488587_PostDataQualityPlan-2.csv', index=False)

# (3) Exploring relationships between feature pairs:

The features that I will look at for pairwise feature interaction are the following continuous features:
- ExternalRiskEstimate
- PercentTradesNeverDelq
- NumTrades90Ever2DerogPubRec
- NumSatisfactoryTrades
- MSinceMostRecentDelq
- NumBank2NatlTradesWHighUtilization
- NetFractionRevolvingBurden
- NetFractionInstallBurden
- NumInstallTradesWBalance
- NumRevolvingTradesWBalance
- NumTotalTrades
- PercentTradesWBalance
- NumTradesOpeninLast12M
- PercentInstallTrades
- AverageMInFile
- MSinceOldestTradeOpen




and the following categorical features:
- MaxDelqEver
- MaxDelq2PublicRecLast12M
- RiskPerformance

The choices here I based on the idea that these particular features can affect the RiskPerformance target feature, due to the reasoning that type and number of trades, length of time of trades, delinquencies, and trades with balances are all highly likely to affect the risk.

### Correlations for the numeric features

In [ ]:
# Correlation matrix using code found on https://stanford.edu/~mwaskom/software/seaborn/examples/many_pairwise_correlations.html
sns.set(style="white")

# Select columns containing continuous data
continuous_columns = df[['ExternalRiskEstimate', 'PercentTradesNeverDelq', 'NumTrades90Ever2DerogPubRec', 
'NumSatisfactoryTrades', 'MSinceMostRecentDelq', 'NumBank2NatlTradesWHighUtilization','NetFractionRevolvingBurden', 
'NetFractionInstallBurden', 'NumInstallTradesWBalance', 'NumRevolvingTradesWBalance', 'NumTotalTrades', 'PercentTradesWBalance',
    'NumTradesOpeninLast12M', 'PercentInstallTrades', 'AverageMInFile', 'MSinceOldestTradeOpen']].columns

# Calculate correlation of all pairs of continuous features
corr = df[continuous_columns].corr()

# Generate a mask for the upper triangle
mask = np.zeros_like(corr, dtype=np.bool)
mask[np.triu_indices_from(mask)] = True

# Set up the matplotlib figure
f, ax = plt.subplots(figsize=(14, 12))

# Generate a custom colormap - blue and red
cmap = sns.diverging_palette(220, 10, as_cmap=True)

# Draw the heatmap with the mask and correct aspect ratio
sns.heatmap(corr, annot=True, mask=mask, cmap=cmap, vmax=1, vmin=-1,
            square=True, xticklabels=True, yticklabels=True,
            linewidths=.5, cbar_kws={"shrink": .5}, ax=ax)
plt.yticks(rotation = 0)
plt.xticks(rotation = 45)

In the following cells are the plots for pairs of continuous features which have been shown to have a high correlation above.

In [ ]:
df.plot(kind='scatter', x='PercentTradesNeverDelq', y='ExternalRiskEstimate')
df.plot(kind='scatter', x='NumSatisfactoryTrades', y='NumTotalTrades')
df.plot(kind='scatter', x='NumBank2NatlTradesWHighUtilization', y='NumRevolvingTradesWBalance')
df.plot(kind='scatter', x='NumBank2NatlTradesWHighUtilization', y='NetFractionRevolvingBurden')
df.plot(kind='scatter', x='PercentTradesWBalance', y='NetFractionRevolvingBurden')
df.plot(kind='scatter', x='AverageMInFile', y='MSinceOldestTradeOpen')

From these plots it is apparent that the stronger the correlation (the closer to 1.0 in the heat map), the more of a linear correlation occurs. The average months in file grows linearly as the months since the oldest trade open increases for example. The number of satisfactory trades verus total number of trades has the highest correlation, and thus is the most linear. This shows that the vast majority of trades are satisfactory, and thus delinquencies are low.

We can also see that the percentage of trades with balance is related to the net fraction revolving burden, showing that revolving trades are generally riskier than installment trades. 

### Categorical feature plots

In [ ]:
MaxDelq2PublicRecLast12M = pd.unique(df.MaxDelq2PublicRecLast12M.ravel())
dfnew = df.copy()
dfnew['percent'] = 0

for i in MaxDelq2PublicRecLast12M:
    count = 1 / dfnew[dfnew.MaxDelq2PublicRecLast12M == i].count()['RiskPerformance']
    index_list = dfnew[dfnew['MaxDelq2PublicRecLast12M'] == i].index.tolist()
    for ind in index_list:
        dfnew.loc[ind, 'percent'] = count * 100
        
group = dfnew[['percent','MaxDelq2PublicRecLast12M','RiskPerformance']].groupby(['MaxDelq2PublicRecLast12M','RiskPerformance']).sum()

my_plot = group.unstack().plot(kind='bar', stacked=True, title="RiskPerformance based on MaxDelq2PublicRecLast12M", figsize=(15,7))

red_patch = mpatches.Patch(color='orange', label='Good')
blue_patch = mpatches.Patch(color='blue', label='Bad')
my_plot.legend(handles=[red_patch, blue_patch], frameon = True)

my_plot.set_xlabel("MaxDelq2PublicRecLast12M")
my_plot.set_ylabel("RiskPerformance")
my_plot.set_ylim([0,100])

#### The results of this bar plot are largely in line with what we would expect: values 0 - 4 means delinquency, 6 means unknown delinquency, and 7 means current and never delinquent. Therefore unsurprisingly, 7 has the highest amount of 'Good' outcomes. 

#### One interesting point is how 90 days delinquent (bar 2) always results in a 'Bad' outcome, whereas 120+ days delinquent (bar 1), which we would expect to be worse, actually has more 'Good' outcomes. This shows that there are other factors at play when deciding a 'Good' or 'Bad' outcome.

In [ ]:
MaxDelqEver = pd.unique(df.MaxDelqEver.ravel())
dfnew = df.copy()
dfnew['percent'] = 0

for i in MaxDelqEver:
    count = 1 / dfnew[dfnew.MaxDelqEver == i].count()['RiskPerformance']
    index_list = dfnew[dfnew['MaxDelqEver'] == i].index.tolist()
    for ind in index_list:
        dfnew.loc[ind, 'percent'] = count * 100
        
group = dfnew[['percent','MaxDelqEver','RiskPerformance']].groupby(['MaxDelqEver','RiskPerformance']).sum()

my_plot = group.unstack().plot(kind='bar', stacked=True, title="RiskPerformance based on MaxDelqEver", figsize=(15,7))

red_patch = mpatches.Patch(color='orange', label='Good')
blue_patch = mpatches.Patch(color='blue', label='Bad')
my_plot.legend(handles=[red_patch, blue_patch], frameon = True)

my_plot.set_xlabel("MaxDelqEver")
my_plot.set_ylabel("RiskPerformance")
my_plot.set_ylim([0,100])

#### Here, bar 3 which is 120+ days delinquent has the highest 'Bad' outcome, whereas bar 8 which is current and never delinquent has the highest 'Good' outcome, as we would expect.

### Continuous-categorical feature plots

In [ ]:
plt.figure()
flierprops = dict(marker='o', markerfacecolor='green', markersize=6,
                  linestyle='none')

df.boxplot(column=['ExternalRiskEstimate'], by=['RiskPerformance'], flierprops=flierprops, figsize=(10,7))
df.boxplot(column=['MSinceOldestTradeOpen'], by=['RiskPerformance'], flierprops=flierprops, figsize=(10,7))
df.boxplot(column=['AverageMInFile'], by=['RiskPerformance'], flierprops=flierprops, figsize=(10,7))

df.boxplot(column=['NumSatisfactoryTrades'], by=['RiskPerformance'], flierprops=flierprops, figsize=(10,7))
df.boxplot(column=['PercentTradesNeverDelq'], by=['RiskPerformance'], flierprops=flierprops, figsize=(10,7))
df.boxplot(column=['NumRevolvingTradesWBalance'], by=['RiskPerformance'], flierprops=flierprops, figsize=(10,7))


df.boxplot(column=['NumBank2NatlTradesWHighUtilization'], by=['RiskPerformance'], flierprops=flierprops, figsize=(10,7))
df.boxplot(column=['PercentTradesWBalance'], by=['RiskPerformance'], flierprops=flierprops, figsize=(10,7))
df.boxplot(column=['MSinceMostRecentDelq'], by=['RiskPerformance'], flierprops=flierprops, figsize=(10,7))

df.boxplot(column=['NumTotalTrades'], by=['RiskPerformance'], flierprops=flierprops, figsize=(10,7))
df.boxplot(column=['NetFractionInstallBurden'], by=['RiskPerformance'], flierprops=flierprops, figsize=(10,7))
df.boxplot(column=['NetFractionRevolvingBurden'], by=['RiskPerformance'], flierprops=flierprops, figsize=(10,7))




In [ ]:
df[df['RiskPerformance'] == 'Good'][['ExternalRiskEstimate']].hist(figsize=(7,7), bins=40)
df[df['RiskPerformance'] == 'Bad'][['ExternalRiskEstimate']].hist(figsize=(7,7), bins=40)

In [ ]:
df[df['RiskPerformance'] == 'Good'][['MSinceOldestTradeOpen']].hist(figsize=(7,7), bins=40)
df[df['RiskPerformance'] == 'Bad'][['MSinceOldestTradeOpen']].hist(figsize=(7,7), bins=40)

In [ ]:
df[df['RiskPerformance'] == 'Good'][['AverageMInFile']].hist(figsize=(7,7), bins=40)
df[df['RiskPerformance'] == 'Bad'][['AverageMInFile']].hist(figsize=(7,7), bins=40)

In [ ]:
df[df['RiskPerformance'] == 'Good'][['NumSatisfactoryTrades']].hist(figsize=(7,7), bins=40)
df[df['RiskPerformance'] == 'Bad'][['NumSatisfactoryTrades']].hist(figsize=(7,7), bins=40)

In [ ]:
df[df['RiskPerformance'] == 'Good'][['PercentTradesNeverDelq']].hist(figsize=(7,7), bins=40)
df[df['RiskPerformance'] == 'Bad'][['PercentTradesNeverDelq']].hist(figsize=(7,7), bins=40)

In [ ]:
df[df['RiskPerformance'] == 'Good'][['NetFractionRevolvingBurden']].hist(figsize=(7,7), bins=40)
df[df['RiskPerformance'] == 'Bad'][['NetFractionRevolvingBurden']].hist(figsize=(7,7), bins=40)

In [ ]:
df[df['RiskPerformance'] == 'Good'][['NumBank2NatlTradesWHighUtilization']].hist(figsize=(7,7), bins=40)
df[df['RiskPerformance'] == 'Bad'][['NumBank2NatlTradesWHighUtilization']].hist(figsize=(7,7), bins=40)

In [ ]:
df[df['RiskPerformance'] == 'Good'][['PercentTradesWBalance']].hist(figsize=(7,7), bins=40)
df[df['RiskPerformance'] == 'Bad'][['PercentTradesWBalance']].hist(figsize=(7,7), bins=40)

In [ ]:
df[df['RiskPerformance'] == 'Good'][['MSinceMostRecentDelq']].hist(figsize=(7,7), bins=40)
df[df['RiskPerformance'] == 'Bad'][['MSinceMostRecentDelq']].hist(figsize=(7,7), bins=40)

## Findings:

From the continuous - continuous data visualization, we can see that most 
trades carried out result in a satisfactory outcome. There is also a strong link between
months since oldest trade open and average month in file, which makes sense as increase as time
progresses. 

We can also see that Number Bank/Natl Trades w high utilization ratio
and revolving trades are strongly correlated, and not installment trades. This signifies that Bank/Natl
trades with balance are most likely going to be revolving trades.

Net fraction revolving burden is correlated highly with percentage of trades with balance, which
seems to be pointing at revolving trades being the riskier trade type.

With categorical - categorical, rather unsurprisingly we see that those who avoid delinquency tend to have better chance at 
having a 'Good' outcome.

In categorical - continuous, a higher external risk estimate is indicative of a good risk performance outcome, with a larger 
number of months since most recent delinquency also serving as an indictor of a 'Good' outcome.

Higher values of net fraction revolving burden and percentage trades with balance are very strong indicators for 
a 'Bad' outcome. Interestingly, the number of satisfactory trades is more loosely linked
to a 'Good' outcome, showing that present revolving burden and trades with balance outweigh the
past 'goodwill' from previous satisfactory outcomes.

These aforementioned features are potential candidates for a predictive model of risk performance.

# Transform, extend or combine the existing features to create new features

#### Something which the data highlighted is that the longer a person has been trading, the more likely there was to be a 'Good' outcome. However, the months since the oldest trade open and months since the most recent trade open were not looked at in combination in our data which is odd, as 'MSinceOldestTradeOpen' - 'MSinceMostRecentTradeOpen' gives the months within which the customer was trading. A customer could have a very old MSinceOldestTradeOpen, but have stopped trading a month later.  As seen from the boxplot below, the longer total months trading, from first to last, means a better chance at a 'Good' outcome.

In [ ]:
df['MonthsTrading'] = df['MSinceOldestTradeOpen'] - df['MSinceMostRecentTradeOpen']

In [ ]:
df.boxplot(column=['MonthsTrading'], by=['RiskPerformance'], flierprops=flierprops, figsize=(10,7))

#### Based on the data seen so far, a high net fraction burden is a good indicator of a 'Bad' outcome. While NetFractionInstallBurden has the greatest sway on the target feature, NetFractionInstallBurden is also an indicator for the target outcome. Therefore, adding the two fractions together gives a feature with a strong correlation towards the target feature; the higher the total net fraction burden, the more likely a 'Bad' outcome will occur. 

In [ ]:
df['TotalNetFractionBurden'] = df['NetFractionInstallBurden'] + df['NetFractionRevolvingBurden']

In [ ]:
df.boxplot(column=['TotalNetFractionBurden'], by=['RiskPerformance'], flierprops=flierprops, figsize=(10,7))

#### From looking at the data, it can be seen that the number of satisfactory trades does not always equal to the total number of trades minus the delinquent trades. As delinquency starts at 30 days in the data dictionary, it can be assumed that any late payments below 30 days aren't counted as delinquent, and are still counted as satisfactory. Thus, a percentage of satisfactory trades feature can more accurately capture the data we have.

In [ ]:
df['PercentSatisTrades'] = (df['NumSatisfactoryTrades']*100) / df['NumTotalTrades']

#### Many of the features such as 'MSinceMostRecentInqexcl7days' have large 'tails' (exponential distribution) and thus a large amount of outliers.  Many machine learning algorithms benefit from having such data normalised. 

#### "Having continuous features in an ABT that cover very different ranges can cause difficulty for some machine learning algorithms.....Normalization techniques can be used to change a continuous feature to fall within a specified range while maintaining the relative differences between the values for the feature." - Fundamentals of Machine Learning For Predictive Data Analytics.

In [ ]:
df['MSinceMostRecentInqexcl7days'] = (df['MSinceMostRecentInqexcl7days'] - df['MSinceMostRecentInqexcl7days'].mean())/df['MSinceMostRecentInqexcl7days'].std()

In [ ]:
df['MSinceMostRecentInqexcl7days'].hist(figsize=(10,10))

In [ ]:
df

In [ ]:
df.to_csv('06488587_added_features.csv', index=False)